In [1]:
from pysat.formula import CNF
import sys
import os
import math

In [2]:
import pysat
pysat.__version__

'0.1.7.dev12'

In [3]:
test_sats_path = 'test sudokus'

In [4]:
def read_DIMACS(path):
    return CNF(from_file=path).clauses
    
    
rules = read_DIMACS(os.path.join(test_sats_path, 'sudoku-rules-9x9.txt'))
rules[:10]

[[111, 112, 113, 114, 115, 116, 117, 118, 119],
 [-111, -112],
 [-111, -113],
 [-111, -114],
 [-111, -115],
 [-111, -116],
 [-111, -117],
 [-111, -118],
 [-111, -119],
 [-112, -113]]

In [5]:
input_mapping = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9, 'A': 10, 'B': 11, "C": 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16}

inverse_input_mapping = {v:k for k, v in input_mapping.items()}
inverse_input_mapping

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: 'A',
 11: 'B',
 12: 'C',
 13: 'D',
 14: 'E',
 15: 'F',
 16: 'G'}

In [6]:
def line2DIMACS(sudoku):
    input_len = len(sudoku)
    table_size = int(math.pow(input_len, 0.5))
    clause_list = []
    for row in range(table_size):
        for column in range(table_size):
            current_line_index = column + table_size * row
            cell_value = sudoku[current_line_index]
            if cell_value != '.':
                clause_list.append(f"{inverse_input_mapping[row+1]}{inverse_input_mapping[column+1]}{cell_value} 0")
    return clause_list, table_size

def line2CNF(line):
    cnf_list, table_size = line2DIMACS(line)
    return CNF(from_string='\n'.join(cnf_list)).clauses, cnf_list, table_size

In [7]:
# with open(os.path.join(test_sats_path, '4x4.txt')) as f:
with open(os.path.join(test_sats_path, '1000 sudokus.txt')) as f:
    sudoku_list = f.readlines()

In [8]:
sudoku_setup_CNF, sudoku_setup_DIMACS, sudoku_size = line2CNF(sudoku_list[3])

In [9]:
from dpll import DPLL

In [10]:
clauses = rules + sudoku_setup_CNF
# clauses = [[1,2], [1], [-3]]

solver = DPLL(clauses, variable_selection_method='random', verbose=0)

In [11]:
# solver.variables_set

In [12]:
# solver.clauses

In [ ]:
%%time

solution = solver.backtrack(solver.clauses, partial_assignment=[], split_literal=tuple())

In [ ]:
solution

In [ ]:
def get_digit(number, n):
    return int(number // 10**n % 10)

import numpy as np
matrix = np.zeros((sudoku_size, sudoku_size), dtype=int)
for clause in sudoku_setup_CNF:
    for item in clause:
        matrix[get_digit(item, 2)-1, get_digit(item, 1)-1] = get_digit(item, 0)

In [ ]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
        
matprint(matrix)

In [ ]:
found_assignments

In [ ]:
found_assignments = sorted(solution[1], key=lambda x: x[0])

matrix = np.zeros((sudoku_size, sudoku_size), dtype=int)
for assignment in found_assignments:
    if assignment[1]:
        item = assignment[0]
        matrix[int(item[0])-1, int(item[1])-1] = int(item[2])
        
matprint(matrix)